<a href="https://colab.research.google.com/github/ferrari-leo/training/blob/main/practical_mab/mab_implementations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
class Environment(object):
  def __init__(self, reward_probabilities, actual_rewards):
    if len(reward_probabilities) != len(actual_rewards):
      raise Exception(f'size of reward probabilities {len(reward_probabilities)} does not match size of actual rewards {len(actual_rewards)}')

    self.reward_probabilities = reward_probabilities
    self.actual_rewards = actual_rewards
    self.k_arms = len(actual_rewards)

  def choose_arm(self, arm):
    if arm < 0 or arm >= self.k_arms:
      raise Exception(f'arm must be between 0 and {self.k_arms - 1}')

    return self.actual_rewards[arm] if np.random.random() < self.reward_probabilities[arm] else 0.0

In [3]:
env = Environment(
    reward_probabilities=[0.62, 0.05, 0.87, 0.49],
    actual_rewards=[1.0, 1.0, 1.0, 1.0]
)

In [4]:
[env.choose_arm(1) for _ in range(10)]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]